In [7]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [8]:
SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

In [9]:
usecase_name = "Flashcard"
project_name = "ShopeeWeb"
main_flow_prompt = """
basic flow:
Step 1: Learner clicks the "Learn New Word" tab on the navigation bar.
Step 2:The system displays a list of available lessons.
Step 3:Learner selects a lesson from the list.
Step 4:The system displays the details of the selected lesson, including the topics associated with it.
Step 5:Learner chooses a topic to study.
Step 6:The system redirects the learner to the corresponding Flashcard page for the selected topic.
Step 7:The system displays the first flashcard of the vocabulary words for the selected topic. The front of the flashcard shows the vocabulary name, pronunciation, and Vietnamese meaning.
Step 8:Learner click on the flashcard to flip it and view the back, which contains illustrations and examples of real-life usage.
Step 9:Learner click the "Next" button to go to the flashcard of the next vocabulary word.
Step 10:If there is a flashcard of the previous word, learner click the "Previous" button to go back to the previous flashcard.
Step 11:Steps 8-11 are repeated for all flashcards in the topic.
Step 12:Once all the flashcards have been studied, the 'Next' button on the last flashcard will redirect system to the 'completion notification' page with a congratulatory message and option buttons to 'continue studying' and 'return to the lesson page'.
Step 13: learner clicks "return to the lesson page" 
Step 14: the system redirect to the lesson page.

"""
prompt_all = """
basic flow:
Step 1: Learner clicks the "Learn New Word" tab on the navigation bar.
Step 2:The system displays a list of available lessons.
Step 3:Learner selects a lesson from the list.
Step 4:The system displays the details of the selected lesson, including the topics associated with it.
Step 5:Learner chooses a topic to study.
Step 6:The system redirects the learner to the corresponding Flashcard page for the selected topic.
Step 7:The system displays the first flashcard of the vocabulary words for the selected topic. The front of the flashcard shows the vocabulary name, pronunciation, and Vietnamese meaning.
Step 8:Learner click on the flashcard to flip it and view the back, which contains illustrations and examples of real-life usage.
Step 9:Learner click the "Next" button to go to the flashcard of the next vocabulary word.
Step 10:If there is a flashcard of the previous word, learner click the "Previous" button to go back to the previous flashcard.
Step 11:Steps 8-11 are repeated for all flashcards in the topic.
Step 12:Once all the flashcards have been studied, the 'Next' button on the last flashcard will redirect system to the 'completion notification' page with a congratulatory message and option buttons to 'continue studying' and 'return to the lesson page'.
Step 13: learner clicks "return to the lesson page" 
Step 14: the system redirect to the lesson page.

Alternative Flows:
Alternative flow 1: Learner presses the 'back' button during flashcard study and presses 'stay'.
At step 7 of the basic flow: Learner presses the 'back button.
Step 8: Exit confirmation dialog box appears, asking "Are you sure you want to exit?"
Step 9: Learner clicks 'stay'.
Go back to step 7 in the basic flow and continue with the steps from step 7.

Alternative flow 2:Learner presses the 'continue studying'
At step 13 of the basic flow: learner clicks 'continue studying'
Step 14: Topic is marked as completed. Vocabulary words are marked as studied.Learner can review completed topics and studied vocabulary later.
Step 15: The system redirects the learner to the corresponding Flashcard page for the continuous topic.

Alternative flow 3: Continuing from Home Page with incomplete topic displayed on the home page
At step 1 of the basic flow: Learner clicks one incomplete topic.
Go back to step 6 in the basic flow and continue with the steps from step 6.

Alternative flow 4: learner tries to click 'Previous' button when on the first flashcard, the button is disabled.
At step 9 of the basic flow: Learner clicks 'Previous' button.
Step 10: the system display message 'there is no previous vocabulary to display'.
Go back to step 9 in the basic flow and continue with the steps from step 9.

Exception flow:
Exception flow 1: Learner presses the 'back' button during flashcard study and presses 'exit'.
At step 7 of the basic flow: Learner presses the 'back' button.
Step 8: Exit confirmation dialog box appears, asking "Are you sure you want to exit?"
Step 9: Learner clicks 'exit'.
Step 10: System returns learner to the topic page and the topic is marked as incomplete topic and be displayed on the home page.
"""
alt_prompt=["""Alternative flow 1: Learner presses the 'back' button during flashcard study and presses 'stay'.
At step 7 of the basic flow: Learner presses the 'back button.
Step 8: Exit confirmation dialog box appears, asking "Are you sure you want to exit?"
Step 9: Learner clicks 'stay'.
Go back to step 7 in the basic flow and continue with the steps from step 7.
""","""
Alternative flow 2:Learner presses the 'continue studying'
At step 13 of the basic flow: learner clicks 'continue studying'
Step 14: Topic is marked as completed. Vocabulary words are marked as studied.Learner can review completed topics and studied vocabulary later.
Step 15: The system redirects the learner to the corresponding Flashcard page for the continuous topic.
""","""
Alternative flow 3: Continuing from Home Page with incomplete topic displayed on the home page
At step 1 of the basic flow: Learner clicks one incomplete topic.
Go back to step 6 in the basic flow and continue with the steps from step 6.
""","""
Alternative flow 4: learner tries to click 'Previous' button when on the first flashcard, the button is disabled.
At step 9 of the basic flow: Learner clicks 'Previous' button.
Step 10: the system display message 'there is no previous vocabulary to display'.
Go back to step 9 in the basic flow and continue with the steps from step 9."""]
exc_prompt = ["""Exception flow 1: Learner presses the 'back' button during flashcard study and presses 'exit'.
At step 7 of the basic flow: Learner presses the 'back' button.
Step 8: Exit confirmation dialog box appears, asking "Are you sure you want to exit?"
Step 9: Learner clicks 'exit'.
Step 10: System returns learner to the topic page and the topic is marked as incomplete topic and be displayed on the home page."""]

In [10]:

for i in range(1,4):
  print("TIME "+str(i))

  for alt in alt_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n"+alt}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    print(main_flow_prompt +"\n"+alt)
    print("\nsub_gpt_response:"+sub_gpt_response +"\n")
  for exc in exc_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n"+exc}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    print(main_flow_prompt +"\n"+exc)
    print("\nsub_gpt_response:"+sub_gpt_response +"\n")

TIME 1

basic flow:
Step 1: Learner clicks the "Learn New Word" tab on the navigation bar.
Step 2:The system displays a list of available lessons.
Step 3:Learner selects a lesson from the list.
Step 4:The system displays the details of the selected lesson, including the topics associated with it.
Step 5:Learner chooses a topic to study.
Step 6:The system redirects the learner to the corresponding Flashcard page for the selected topic.
Step 7:The system displays the first flashcard of the vocabulary words for the selected topic. The front of the flashcard shows the vocabulary name, pronunciation, and Vietnamese meaning.
Step 8:Learner click on the flashcard to flip it and view the back, which contains illustrations and examples of real-life usage.
Step 9:Learner click the "Next" button to go to the flashcard of the next vocabulary word.
Step 10:If there is a flashcard of the previous word, learner click the "Previous" button to go back to the previous flashcard.
Step 11:Steps 8-11 are r